# Data Kiosk with SP-API SDK: Generate, Run, and Visualize Queries

## Setup
### 🧰 Installing Selling Partner API SDK

The **Amazon Selling Partner API SDK** is an official Amazon library that simplifies integration with Amazon’s SP-API. It handles things like authentication, request signing, and response parsing — so you can focus on building your solution instead of dealing with low-level API mechanics.

With this SDK, you can easily access endpoints like Orders, Listings, Reports, Feeds, and **Buy Shipping** using clean, structured Python code.

📚 Learn more or explore the source on GitHub: https://github.com/amzn/selling-partner-api-sdk/tree/main


📦 To install the SDK, run the below block 👇

In [ ]:
pip install amzn-sp-api==1.2.0

## Let's Data Kiosk!

Data Kiosk lets you submit GraphQL queries from a variety of schemas to help manage selling partner businesses. You can use the following SDK to call Data Kiosk API.


### 🔍 What is Data Kiosk?

It’s your all-access pass to query Amazon datasets like sales, traffic, and profitability using GraphQL.

You can:

- Submit custom queries
- Check their status
- Download results
  
🙌 Time to get seller data with the power of GraphQL!



---

### 🐛 Configuring the SDK


Luckily the Amazon Solution Architect team created a mock server that can replicate the API responses without having credentials.

Below you will find `mock_oauth_endpoint` and `mock_endpoint` variables that can be used to skip the credentials.

In [ ]:
from spapi.models.datakiosk_v2023_11_15 import GetDocumentResponse, CreateQueryResponse, GetQueriesResponse, Query
from spapi import SPAPIConfig, SPAPIClient, ApiException, QueriesApi, ProductPricingV2022Api, ListingsApi
import json

# Configuration for mock SP-API
mock_oauth_endpoint = "INSERT_OAUTH_MOCK_ENDPOINT"
mock_endpoint = "INSERT_MOCK_ENDPOINT"

config = SPAPIConfig(
    client_id="YOUR_CLIENT_ID",
    client_secret="YOUR_CLIENT_SECRET",
    refresh_token="YOUR_REFRESH_TOKEN",
    region="NA"
)

# Initializing MOCK SP-API SDK Client
client = SPAPIClient(config, endpoint=mock_endpoint, oauth_endpoint=mock_oauth_endpoint)

# Uncomment the below line if you want to test in prod and not make MOCK calls
# client = SPAPIClient(config)

# Initializing the Data Kiosk API Client
data_kiosk_api = QueriesApi(client.api_client)

print("Connected to SP-API")


---

### 🚶‍♂️ Steps to Get Data Kiosk Reports

#### 🖥️ Step 1: Paste and create Query Below

💡 Use the [Data Kiosk Schema Explorer UI](https://sellercentral.amazon.com/datakiosk-schema-explorer?schema=analytics_salesAndTraffic_2024_04_24) to build your perfect query, for now, you can use the one we built for you here:

In [ ]:
# This is a sample query - please update with your own query
# Make sure to use backspaces with the quotation marks
graphql_query_str = (
    "query {analytics_salesAndTraffic_2024_04_24 "
    "{salesAndTrafficByAsin(aggregateBy:CHILD,endDate:\"2025-06-06\",startDate:\"2025-03-01\",marketplaceIds:[\"ATVPDKIKX0DER\"]) "
    "{sales {orderedProductSales {amount currencyCode} totalOrderItems unitsOrdered} "
    "traffic {pageViews buyBoxPercentage} sku childAsin}}}"
)

graphql_query = {"query": graphql_query_str}

# Tell Amazon to start processing your query
create_query_response = data_kiosk_api.create_query(body=graphql_query)
print("✅ Query submitted! Response:", create_query_response)

---

#### 🚀 Step 2: Send the Query
Let Amazon know you’re ready to roll by calling `create_query`.

In [ ]:
# Tell Amazon to start processing your query
create_query_response = data_kiosk_api.create_query(body=graphql_query)
print("✅ Query submitted! Response:", create_query_response)

---

#### ⏳ Step 3: Check If Your Data Is Ready

You can either:

- Keep checking with `get_query`, or
- 🧠 Be smart _well-architected_ and subscribe to the [DATA_KIOSK_QUERY_PROCESSING_FINISHED](https://developer-docs.amazon.com/sp-api/docs/data-kiosk-notification) notification for automatic updates.

Here, we’ll use polling just once for simplicity - if data is ready a `dataDocumentId` attribute will be reutrned:

In [ ]:
# Check the status of your query
get_query_response = data_kiosk_api.get_query(query_id=create_query_response.query_id)
print("⏱️ Query status:", get_query_response)

---

#### 📦 Step 4: Get Your Data!

Once the query is ready, you’ll receive a `dataDocumentId`. Use it to grab the results with `get_document` endpoint.

In [ ]:
# Now let's get the actual data!
get_document = data_kiosk_api.get_document(document_id=get_query_response.data_document_id)
print("📊 Here's your data:", get_document)

---

### 📊 Visualize Your Data


🧠 Let's get our Data Kiosk response and interpret it into a nice dashboard.

Below, we’ll:
- Parse the economics data from `get_document_response`
- Build a DataFrame
- Plot net product sales and units sold per SKU

Make sure you’ve run the cell above so `get_document_response` is available.

🚨 Update this code to plot the metrics that make sense based on the query you submitted


In [ ]:
import ast
import pandas as pd
import matplotlib.pyplot as plt

# Safely convert the string to a dictionary
parsed_document = ast.literal_eval(get_document.document_url)

# Extract the economics data
economics_data = parsed_document["data"]["analytics_economics_2024_03_15"]["economics"]

# Build a DataFrame
df = pd.DataFrame([
    {
        "MSKU": item["msku"],
        "Start Date": item["startDate"],
        "End Date": item["endDate"],
        "Net Product Sales ($)": item["sales"]["netProductSales"]["amount"],
        "Units Sold": item["sales"]["netUnitsSold"]
    }
    for item in economics_data
])

# Show the raw data
print("🧾 Parsed Economics Data:")
display(df)

# Plotting
fig, ax1 = plt.subplots(figsize=(10, 6))

# Bar chart for Net Product Sales
ax1.bar(df["MSKU"], df["Net Product Sales ($)"], label='Net Product Sales ($)')
ax1.set_ylabel("Sales ($)", fontsize=12)
ax1.set_title("Net Product Sales vs Units Sold by MSKU", fontsize=14)
ax1.set_xlabel("MSKU", fontsize=12)
ax1.legend(loc="upper left")

# Line chart for Units Sold
ax2 = ax1.twinx()
ax2.plot(df["MSKU"], df["Units Sold"], color='orange', marker='o', label='Units Sold')
ax2.set_ylabel("Units Sold", fontsize=12)
ax2.legend(loc="upper right")

plt.tight_layout()
plt.show()


---